In [21]:
"""Example of long-only portfolio among DOW-30 components.

Monthly rebalance, backtest spans from the start of data of the
components of the current index (in the 60s), the other stocks
enter the backtest as times goes on. It ends today.

This uses an explicit loop to create Multi Period Optimization
policies with a grid of values for the risk term multiplier
and the transaction cost term multiplier. (We don't use the
holding cost term because the portfolio is long-only.)

All result objects are collected, and then the one with
largest Sharpe ratio, and the one with largest growth rate,
are shown.

Finally, we show the effect of using symbolic hyper-parameters,
:class:`cvxportfolio.Gamma`, as multipliers of the risk and transaction
cost terms. We can optimize on those explicitely, by finding the values
that maximize some back-test metric (in this case, profit).

You can run this with this command from the parent directory:

.. code-block:: bash

    python -m examples.dow30
"""

# Uncomment the logging lines to get online information
# from the parallel backtest routines

# import logging
# logging.basicConfig(level=logging.INFO)
# log=logging.getLogger('=>')

import os

import matplotlib.pyplot as plt
import numpy as np

import cvxportfolio as cvx

from universes import DOW30

sim = cvx.StockMarketSimulator(DOW30, trading_frequency='monthly')

def make_policy(gamma_trade, gamma_risk):
    """Build MPO policy given risk and trans. cost multipliers.

    :param gamma_trade: Transaction cost multiplier.
    :type gamma_trade: float or int
    :param gamma_risk: Risk model multiplier.
    :type gamma_risk: float or int
    :return: Multi-period optimization policy with given
        hyper-parameter values.
    :rtype: cvxportfolio.Policy
    """
    return cvx.MultiPeriodOptimization(cvx.ReturnsForecast()
        - gamma_risk * cvx.FactorModelCovariance(num_factors=10)
        - gamma_trade * cvx.StocksTransactionCost(),
        [cvx.LongOnly(), cvx.LeverageLimit(1)],
        planning_horizon=6, solver='ECOS')

keys = [(gamma_trade, gamma_risk) for
    gamma_trade in np.array(range(10))/10 for gamma_risk in [.5, 1, 2, 5, 10]]
ress = sim.backtest_many([make_policy(*key) for key in keys])


print('\n\nLARGEST SHARPE RATIO')
idx = np.argmax([el.sharpe_ratio for el in ress])

print('gamma_trade and gamma_risk')
print(keys[idx])

print('result')
print(ress[idx])

ress[idx].plot()
# we use this to save plots for the documentation
if 'CVXPORTFOLIO_SAVE_PLOTS' in os.environ:
    plt.savefig('dow30_largest_sharpe_ratio.png')

print('\n\nLARGEST GROWTH RATE')
idx = np.argmax([el.growth_rates.mean() for el in ress])

print('gamma_trade and gamma_risk')
print(keys[idx])

print('result')
print(ress[idx])

ress[idx].plot()
if 'CVXPORTFOLIO_SAVE_PLOTS' in os.environ:
    plt.savefig('dow30_largest_growth_rate.png')

print('\n\nUNIFORM (1/N) ALLOCATION FOR COMPARISON')
result_uniform = sim.backtest(cvx.Uniform())

print('result_uniform')
print(result_uniform)

result_uniform.plot()
if 'CVXPORTFOLIO_SAVE_PLOTS' in os.environ:
    plt.savefig('dow30_uniform.png')

print('\n\nHYPER-PARAMETER OPTIMIZATION')
policy = cvx.MultiPeriodOptimization(cvx.ReturnsForecast()
        - cvx.Gamma() * cvx.FactorModelCovariance(num_factors=10)
        - cvx.Gamma() * cvx.StocksTransactionCost(),
        [cvx.LongOnly(), cvx.LeverageLimit(1)],
        planning_horizon=6, solver='ECOS')
sim.optimize_hyperparameters(policy, objective='profit')
result_hyperparameter_optimized = sim.backtest(policy)

print('result_hyperparameter_optimized')
print(result_hyperparameter_optimized)

result_hyperparameter_optimized.plot()

if 'CVXPORTFOLIO_SAVE_PLOTS' in os.environ:
    plt.savefig('dow30_hyperparameter_optimized.png')
else:
    plt.show()

Updating data

INFO:cvxportfolio.data.market_data:Updating AAPL with YahooFinance.


.

INFO:cvxportfolio.data.symbol_data:YahooFinance is trying to load AAPL with pickle backend from /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.symbol_data:Downloading AAPL from YahooFinance
INFO:cvxportfolio.data.symbol_data:Downloading from the start.
INFO:cvxportfolio.data.symbol_data:YahooFinance is storing AAPL with pickle backend in /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.symbol_data:YahooFinance is trying to load AAPL with pickle backend from /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.market_data:Updating AMGN with YahooFinance.


.

...

.

INFO:cvxportfolio.data.symbol_data:YahooFinance is trying to load WBA with pickle backend from /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.symbol_data:Downloading WBA from YahooFinance
INFO:cvxportfolio.data.symbol_data:Downloading from the start.
INFO:cvxportfolio.data.symbol_data:YahooFinance("WBA") has non-positive open prices on timestamps: DatetimeIndex(['1980-03-17 14:30:00+00:00', '1980-03-18 14:30:00+00:00',
               '1980-03-19 14:30:00+00:00', '1980-03-20 14:30:00+00:00',
               '1980-03-21 14:30:00+00:00', '1980-03-24 14:30:00+00:00',
               '1980-03-25 14:30:00+00:00', '1980-03-26 14:30:00+00:00',
               '1980-03-27 14:30:00+00:00', '1980-03-28 14:30:00+00:00',
               ...
               '1985-06-17 13:30:00+00:00', '1985-06-18 13:30:00+00:00',
               '1985-06-19 13:30:00+00:00', '1985-06-20 13:30:00+00:00',
               '1985-06-21 13:30:00+00:00', '1985-06-24 13:30:00+00:00',
               '1985-06-2

.

INFO:cvxportfolio.data.symbol_data:YahooFinance is trying to load WMT with pickle backend from /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.symbol_data:Downloading WMT from YahooFinance
INFO:cvxportfolio.data.symbol_data:Downloading from the start.
INFO:cvxportfolio.data.symbol_data:YahooFinance is storing WMT with pickle backend in /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.symbol_data:YahooFinance is trying to load WMT with pickle backend from /Users/edkwang/cvxportfolio_data/YahooFinance
INFO:cvxportfolio.data.symbol_data:Fred is trying to load DFF with pickle backend from /Users/edkwang/cvxportfolio_data/Fred
INFO:cvxportfolio.data.symbol_data:Downloading DFF from Fred


INFO:cvxportfolio.data.symbol_data:Fred is storing DFF with pickle backend in /Users/edkwang/cvxportfolio_data/Fred
INFO:cvxportfolio.data.symbol_data:Fred is trying to load DFF with pickle backend from /Users/edkwang/cvxportfolio_data/Fred
INFO:cvxportfolio.data.market_data:Masking internal DownloadedMarketData dataframes.
INFO:cvxportfolio.simulator:Trying to load cache from disk...
INFO:cvxportfolio.cache:Cache not found!
INFO:cvxportfolio.simulator:Portfolio value at time 1963-02-01 14:30:00: 1000000.0
INFO:cvxportfolio.simulator:Trying to load cache from disk...
INFO:cvxportfolio.simulator:Evaluating the policy at time 1963-02-01 14:30:00
INFO:cvxportfolio.cache:Cache not found!
INFO:cvxportfolio.simulator:Portfolio value at time 1963-02-01 14:30:00: 1000000.0
INFO:cvxportfolio.simulator:Trying to load cache from disk...
INFO:cvxportfolio.simulator:Evaluating the policy at time 1963-02-01 14:30:00
INFO:cvxportfolio.cache:Cache not found!
INFO:cvxportfolio.simulator:Trying to load 